In [4]:
import numpy as np
import os
from os.path import join as pjoin
import nibabel as nib
from utils import save2cifti, get_roi_data

In [2]:
wk_dir = '/nfs/z1/userhome/zzl-xsk/workingdir/nsdretinotopy'
# wk_dir= pjoin(wk_dir, 'NSD-mfm')
ret_dir = pjoin(wk_dir, 'build/retinoparams')
map_dir = pjoin(wk_dir, 'anal/brainmap')
os.makedirs(map_dir, exist_ok=True)

In [5]:
sub = 'sub-30'#'subj01'
layername = 'raw-googlenet-conv2'
params_file = f'{sub}_layer-googlenet-conv2_params.npy'
rfparams = np.load(f'/nfs/z1/userhome/zzl-xsk/Featurespace/build/retinoparams/{sub}_layer-{layername}_params.npy', allow_pickle=True)[0]
n_vertices = 59412
dprf_params = np.zeros((n_vertices, 4))
# dprf_params[:,0] = rfparams['ang']
# dprf_params[:,1] = rfparams['ecc']
# dprf_params[:,2] = rfparams['rfsize']
# dprf_params[:,3] = rfparams['R2']
maps_name = list(rfparams.keys())
for i_index, index in enumerate(maps_name):
    dprf_params[:,i_index] = rfparams[index]
bfiletemplate = nib.load('/nfs/z1/userhome/GongZhengXin/NVP/NaturalObject/data/code/nodretinotopy/mfm_locwise_fullpipeline/template.dtseries.nii')
brain_models = bfiletemplate.header.get_index_map(1).brain_models
surface_brain_models = [bm for bm in brain_models if bm.model_type == 'CIFTI_MODEL_TYPE_SURFACE']
save2cifti(pjoin(map_dir, f'{sub}_layer-{layername}_params.dscalar.nii'),  
            dprf_params.transpose(), surface_brain_models, map_names=maps_name[0:4])


In [17]:
rfparams.keys()

dict_keys(['ecc', 'ang', 'rfsize', 'R2'])

In [20]:
colors = ['#c7522a', '#ffa600', '#008585', '#194a7a']
prf = nib.load('/nfs/z1/userhome/GongZhengXin/NVP/NaturalObject/data/code/nodretinotopy/mfm_locwise_fullpipeline/anal/brainmap/masked_retinotopy/nod_dnn-mean-prf.dscalar.nii').get_fdata()
prfr2 = prf[-1]
voxels = np.where(np.isnan(prfr2)==0)[0]
v1_voxels = np.array([ _ for _ in np.where(get_roi_data(None, 'V1')==1)[0] if _ in voxels])
v2_voxels = np.array([ _ for _ in  np.where(get_roi_data(None, 'V2')==1)[0] if _ in voxels])
v3_voxels = np.array([ _ for _ in  np.where(get_roi_data(None, 'V3')==1)[0] if _ in voxels])
v4_voxels = np.array([ _ for _ in  np.where(get_roi_data(None, 'V4')==1)[0] if _ in voxels])

roi_label = (np.zeros_like(prfr2[np.newaxis, :]))
roi_label[0][v1_voxels] = 1
roi_label[0][v2_voxels] = 2
roi_label[0][v3_voxels] = 3
roi_label[0][v4_voxels] = 4

mapnames = []
label_tables = []
labels_name = {0:'null' ,1: 'select-V1', 2:'select-V2', 3: 'select-V3', 4:'select-V4'}
labels_rgba = {0 : (0,0,0,0),
            1: (199/255,82/255,42/255,1),
          2: (255/255, 166/255, 0, 1),
          3:(0, 133/255, 133/255, 1),
          4:(25/255, 74/255, 122/255, 1)}

mapnames.append('selection')
lbl_tb = nib.cifti2.Cifti2LabelTable()
for key in np.unique(roi_label):
    key = int(key)
    lbl_tb[key] = nib.cifti2.Cifti2Label(key, labels_name[key], *labels_rgba[key])
label_tables.append(lbl_tb)

bfiletemplate = nib.load('/nfs/z1/userhome/GongZhengXin/NVP/NaturalObject/data/code/nodretinotopy/mfm_locwise_fullpipeline/template.dtseries.nii')
brain_models = bfiletemplate.header.get_index_map(1).brain_models
surface_brain_models = [bm for bm in brain_models if bm.model_type == 'CIFTI_MODEL_TYPE_SURFACE']
save2cifti(pjoin('/nfs/z1/userhome/GongZhengXin/NVP/NaturalObject/data/code/nodretinotopy/mfm_locwise_fullpipeline/anal/brainmap/draw', \
            f'nod-selection-privis.dlabel.nii'),  
            roi_label, surface_brain_models, map_names=mapnames, label_tables=label_tables)


In [16]:
mapnames,roi_label.shape

(['selection'], (59412,))